In [1]:
import os
import sys
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
# IMPORTANT: Ensure this order matches the class_indices used during training.
# By default, Keras flow_from_directory sorts classes alphabetically.
CLASS_NAMES = [
    "cocci",
    "healthy",
    "ncd",
    "pcrcocci",
    "pcrhealthy",
    "pcrncd",
    "pcrsalmo",
]

# Optional: nicer display names for UI/outputs
DISPLAY_NAMES = {
    "cocci": "Coccidiosis",
    "healthy": "Healthy",
    "ncd": "Newcastle disease",
    "pcrcocci": "PCR Coccidiosis",
    "pcrhealthy": "PCR Healthy",
    "pcrncd": "PCR Newcastle disease",
    "pcrsalmo": "PCR Salmonella",
}

In [3]:
class PredictionPipeline:
    def __init__(self, filename: str):
        self.filename = filename
        self.model_path = os.path.join("artifacts", "training", "model.keras")
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found at {self.model_path}")
        self.model = load_model(self.model_path, compile=False)

    def _preprocess(self, img_path: str):
        # Match training preprocessing: 224x224 and rescale 1./255
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = x / 255.0
        x = np.expand_dims(x, axis=0)  # (1, 224, 224, 3)
        return x

    def predict(self):
        x = self._preprocess(self.filename)

        # Softmax probabilities
        probs = self.model.predict(x, verbose=0)[0]  # shape: (7,)
        pred_idx = int(np.argmax(probs))
        pred_key = CLASS_NAMES[pred_idx]
        pred_display = DISPLAY_NAMES.get(pred_key, pred_key)
        pred_conf = float(probs[pred_idx])

        return {
            "filename": os.path.basename(self.filename),
            "predicted_key": pred_key,
            "predicted_name": pred_display,
            "confidence": round(pred_conf, 4),
            "class_probabilities": {
                key: round(float(p), 6) for key, p in zip(CLASS_NAMES, probs)
            },
        }

In [4]:
if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python predict.py <image_path>")
        sys.exit(1)
    predictor = PredictionPipeline(sys.argv[1])
    result = predictor.predict()
    print(result)

FileNotFoundError: Model not found at artifacts\training\model.keras

In [ ]:
import os
import sys
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# IMPORTANT: Ensure this order matches the class_indices used during training.
# By default, Keras flow_from_directory sorts classes alphabetically.
CLASS_NAMES = [
    "cocci",
    "healthy",
    "ncd",
    "pcrcocci",
    "pcrhealthy",
    "pcrncd",
    "pcrsalmo",
]

# Optional: nicer display names for UI/outputs
DISPLAY_NAMES = {
    "cocci": "Coccidiosis",
    "healthy": "Healthy",
    "ncd": "Newcastle disease",
    "pcrcocci": "PCR Coccidiosis",
    "pcrhealthy": "PCR Healthy",
    "pcrncd": "PCR Newcastle disease",
    "pcrsalmo": "PCR Salmonella",
}

class PredictionPipeline:
    def __init__(self, filename: str):
        self.filename = filename
        self.model_path = os.path.join("artifacts", "training", "model.keras")
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found at {self.model_path}")
        self.model = load_model(self.model_path, compile=False)

    def _preprocess(self, img_path: str):
        # Match training preprocessing: 224x224 and rescale 1./255
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = x / 255.0
        x = np.expand_dims(x, axis=0)  # (1, 224, 224, 3)
        return x

    def predict(self):
        x = self._preprocess(self.filename)

        # Softmax probabilities
        probs = self.model.predict(x, verbose=0)[0]  # shape: (7,)
        pred_idx = int(np.argmax(probs))
        pred_key = CLASS_NAMES[pred_idx]
        pred_display = DISPLAY_NAMES.get(pred_key, pred_key)
        pred_conf = float(probs[pred_idx])

        return {
            "filename": os.path.basename(self.filename),
            "predicted_key": pred_key,
            "predicted_name": pred_display,
            "confidence": round(pred_conf, 4),
            "class_probabilities": {
                key: round(float(p), 6) for key, p in zip(CLASS_NAMES, probs)
            },
        }

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python predict.py <image_path>")
        sys.exit(1)
    predictor = PredictionPipeline(sys.argv[1])
    result = predictor.predict()
    print(result)
